In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator


import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical

from keras.models import Sequential,load_model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
np.random.seed(1)

In [16]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './images2/train',
        target_size = (300,300),
        batch_size=400,
        color_mode ='rgb',
        classes = ['food','interior', 'exterior'],
        shuffle =True,
        seed = 42,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './images2/test',
        classes =[],
        target_size=(300,300),
        batch_size=100,
        color_mode ='rgb',
        shuffle =True,
        seed = 42,
        class_mode='categorical')

Found 36000 images belonging to 3 classes.
Found 9000 images belonging to 3 classes.


In [17]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(300,300,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, (3, 3), activation='selu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(32, activation='selu'))
model.add(Dense(32, activation='selu'))
model.add(Dense(3, activation='softmax'))

# 3. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=5,
        validation_data=test_generator,
        validation_steps=20)

model.save('model-6')

# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=10)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=10)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

Epoch 1/5
20/20 [==============================] - 1337s 67s/step - loss: 1.0853 - accuracy: 0.3988 - val_loss: 1.0553 - val_accuracy: 0.4150
Epoch 2/5
20/20 [==============================] - 1291s 65s/step - loss: 1.0363 - accuracy: 0.4510 - val_loss: 1.0137 - val_accuracy: 0.4745
Epoch 3/5
20/20 [==============================] - 1280s 64s/step - loss: 1.0074 - accuracy: 0.4789 - val_loss: 0.9903 - val_accuracy: 0.4990
Epoch 4/5
20/20 [==============================] - 1264s 63s/step - loss: 0.9734 - accuracy: 0.5160 - val_loss: 0.9689 - val_accuracy: 0.5205
Epoch 5/5
20/20 [==============================] - 1296s 65s/step - loss: 0.9512 - accuracy: 0.5394 - val_loss: 0.9464 - val_accuracy: 0.5485
INFO:tensorflow:Assets written to: model-6\assets
-- Evaluate --
accuracy: 56.00%
-- Predict --
{'0_food': 0, '1_interior': 1, '2_exterior': 2}
[[0.229 0.357 0.414]
 [0.464 0.322 0.214]
 [0.257 0.258 0.484]
 ...
 [0.305 0.354 0.341]
 [0.682 0.173 0.146]
 [0.513 0.267 0.219]]


In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 147, 147, 16)      4624      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 73, 73, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 85264)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2728480   
_________________________________________________________________
dense_4 (Dense)              (None, 32)               

In [18]:
def predict_image_sample(model,X_test, y_test, test_id= -1 ):
    if test_id <0 :
        from random import randrange
        test_sample_id = randrange(10000)
    else:
        test_sample_id = test_id
    
    test_image =X_test[test_sample_id]
#     plt.imshow(test_images[123])

    
    test_image = test_image.reshape(1,300,300,3)
    
    y_actual = y_test[test_sample_id].tolist().index(1)
    print('y_actual number = ', y_actual)
    
    y_pred = model.predict(test_image)
    print("y_pred = ", y_pred)
    
    y_pred_num = np.argmax(y_pred, axis = 1)[0]
    print('y_pred number = ', y_pred_num)
    return y_actual , y_pred_num


In [19]:
def test_load_data():
    train_images = []       
    train_labels = []
    shape = (300,300)  
    train_path = './image_test'

    for classes in ['f','i','e']:
        for filename in os.listdir('./image_test'):
            if classes == filename[0]:
                img = cv2.imread(os.path.join(train_path,filename))
                train_labels.append(filename[2:0:-1])

                # Resize all images to a specific shape
                #img = cv2.resize(img,shape)

                train_images.append(img)
            else:
                pass

    # Converting labels into One Hot encoded sparse matrix

    # 음식, 실내 ,실외 순으로 들어가기 위해 index 처리
    # train_labels = sorted(train_labels)
    # print(train_labels)

    train_labels = pd.get_dummies((train_labels)).values
    # print(train_labels)
    # Converting train_images to array
    train_images = np.array(train_images)

    # Splitting Training data into train and validation dataset

    train_images ,train_labels= shuffle(train_images ,train_labels, random_state = 42) 

    return train_images, train_labels

In [21]:
if __name__ == '__main__' :
    n = 0
    
    X_test , y_test = test_load_data()
    model = load_model('model-6')
    for i in range(185):
        y_actual , y_pred_num = predict_image_sample(model,X_test, y_test,test_id = i)
        if y_actual == y_pred_num:
            n += 1 
        else : pass
print('전체 횟수 :', i,' '*10,'예측 성공 횟수',n,'\n'*2 ,'정확도 = ', (n/i)*100)
    
    

ValueError: cannot reshape array of size 782502 into shape (1,300,300,3)